In [0]:
%sql

CREATE CATALOG IF NOT EXISTS mini_dolar_futuro;
USE CATALOG mini_dolar_futuro;
CREATE SCHEMA IF NOT EXISTS bronze;
CREATE SCHEMA IF NOT EXISTS silver;
CREATE SCHEMA IF NOT EXISTS gold;


In [0]:
%pip install psycopg2-binary
dbutils.library.restartPython()

In [0]:
import psycopg2
from pyspark.sql.types import *
## OBS: Por Questoes de segurança deixei o password em branco juntamente com o host, caso queira rodar o código substitua por seus dados.

HOST = "colque seu host aqui" 
PORT = "5432"
DATABASE = "postgres"
USER = "postgres.vohkldvhrzlmcslgyfzr" 
PASSWORD = "coloque sua senha aqui" 

source_table = "Wdofut" 
catalog_name = "mini_dolar_futuro"
target_table_name = f"{catalog_name}.bronze.wdofut_raw"

conn = psycopg2.connect(
    host=HOST, port=PORT, database=DATABASE, user=USER, password=PASSWORD, sslmode="require"
)
cursor = conn.cursor()
cursor.execute(f"SELECT * FROM \"{source_table}\" ORDER BY \"data\"")
data = cursor.fetchall()
cursor.close()
conn.close()

cleaned_data = []

for row in data:
    row_list = list(row)
    for i in range(1, 5): 
        value = row_list[i]
        if value is None:
            continue
        value_str = str(value)
        if ',' in value_str:
             value_str = value_str.replace(',', '.')
        row_list[i] = float(value_str) 
    
    cleaned_data.append(tuple(row_list))

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

schema = StructType([
    StructField("data", StringType(), True), 
    StructField("Abertura", DoubleType(), True),
    StructField("Maxima", DoubleType(), True),
    StructField("Minima", DoubleType(), True),
    StructField("Fechamento", DoubleType(), True)
])

df_raw = spark.createDataFrame(cleaned_data, schema=schema)

spark.catalog.setCurrentCatalog(catalog_name) 

(df_raw.write
  .format("delta")
  .mode("overwrite")
  .saveAsTable(target_table_name)
)